 `Pair Programming ETL III transformación II`


En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [14]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda
import requests
import pandas as pd
import numpy as np
df_paises= pd.read_csv('../Datos/attacks_limpieza_IV.csv', index_col= 0)
df_paises = df_paises[df_paises.country.isin(['usa', 'australia', 'new zealand', 'south africa', 'papua new guinea'])]

Creamos la clase

In [15]:
class Desempaquetado:
    def __init__(self, pais, latitud, longitud):
        self.pais= pais
        self.latitud=latitud
        self.longitud=longitud

    def llamada_api(self):

        df_todo = pd.DataFrame()
        url =  f'http://www.7timer.info/bin/api.pl?lon={self.longitud}5&lat=-{self.latitud}6&product=meteo&output=json'
        response=requests.get(url=url)
        df = pd.json_normalize(response.json()['dataseries'])
        df['country'] = f"{self.pais}"
        df_todo = pd.concat([df, df_todo], axis = 0)

        return df_todo


    def limpiando(self, df):
        # df['wind_profile']= df['wind_profile'].apply(ast.literal_eval)

        # df['rh_profile']= df['rh_profile'].apply(ast.literal_eval)
        x = df['rh_profile'].apply(pd.Series)
        x[0].apply(pd.Series)['layer'][0]

        for i in range(len(x.columns)):

            # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string
            nombre = "rh_" + str(x[i].apply(pd.Series)['layer'][0])

            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["rh"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima.
            df.insert(i, nombre, valores)

            y = df['wind_profile'].apply(pd.Series)
            y[0].apply(pd.Series)['layer'][0]
            y


        for i in range(len(y.columns)):

            # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string
            nombre = "wi_speed" + str(y[i].apply(pd.Series)['layer'][0])
            valores = list(y[i].apply(pd.Series)["speed"])
            df.insert(i, nombre, valores)

            nombre2 = "wi_direction" + str(y[i].apply(pd.Series)['layer'][0])
            valores2 = list(y[i].apply(pd.Series)["direction"])
            df.insert(i, nombre2, valores2)
              
        df_mean = df.groupby("country").mean()
        df_mean=df_mean.rename(columns={'wind10m.direction': 'wind10m_direction',
               'wind10m.speed': 'wind10m_speed'})
        
        df = pd.merge(left = df_paises, right= df_mean, how= "left", left_on= "country", right_on= "country")
        df.to_pickle('../Datos/datos_clima.pkl')
        df.to_csv('../Datos/datos_clima.csv')
       
        return df

In [16]:
desemp=Desempaquetado('usa',39.7837304, -100.445882)

In [17]:
df_limpiando=desemp.llamada_api()

In [18]:
df_limpiando

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 10}, {'layer': '900m...","[{'layer': '950mb', 'direction': 295, 'speed':...",10,10,9,1030,none,1,0,335,2,usa
1,6,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 290, 'speed':...",10,10,9,1029,none,1,0,340,3,usa
2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 305, 'speed':...",10,10,8,1029,none,1,0,340,3,usa
3,12,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 10}, {'layer': '900m...","[{'layer': '950mb', 'direction': 335, 'speed':...",10,10,9,1029,none,1,0,340,3,usa
4,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 325, 'speed':...",10,6,11,1027,none,1,0,340,4,usa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,180,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 15}, {'layer': '900m...","[{'layer': '950mb', 'direction': 245, 'speed':...",11,6,15,1015,rain,4,0,155,3,usa
60,183,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 185, 'speed':...",11,6,15,1014,none,4,0,180,3,usa
61,186,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 185, 'speed':...",11,6,14,1015,none,4,0,180,3,usa
62,189,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 230, 'speed':...",10,6,13,1015,none,4,0,205,2,usa


In [19]:
df_final=desemp.limpiando(df_limpiando)
df_final

,year,type,country,area,location,activity,name,time,species,siglo,...,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m_speed
0,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,18h00,White shark,siglo - XXI,...,-9999.0,-9999.0,-9999.0,10.703125,7.53125,11.59375,1020.78125,2.765625,0.0,3.796875
1,2018,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,17h00,"Bull shark, 6'",siglo - XXI,...,-9999.0,-9999.0,-9999.0,10.703125,7.53125,11.59375,1020.78125,2.765625,0.0,3.796875
2,2018,Provoked,australia,Queensland,Cairns Aquarium,Feeding sharks,male,Morning,Grey reef shark,siglo - XXI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,Unprovoked,australia,Western Australia,"Surfers Point, Prevelly",Surfing,Rob Bruce,09h30,3 m shark,siglo - XXI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,Unprovoked,south africa,Western Cape Province,"Robberg Beach, Plettenberg Bay",Paddle-skiing,male,09h40,"White shark, 2.5 m",siglo - XXI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,1997,Unprovoked,australia,NaN,NaN,Fishing,fisherman,Unknow,NaN,Unknow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1351,1997,Unprovoked,australia,New South Wales,,Swimming,Arab boy,Unknow,NaN,Unknow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1352,1997,Unprovoked,australia,Western Australia,Roebuck Bay,Diving,male,Unknow,NaN,Unknow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1353,1997,Unprovoked,australia,Western Australia,NaN,Pearl diving,Ahmun,Unknow,NaN,Unknow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
